# The Chat Format

In this notebook, you will explore how you can utilize the chat format to have extended conversations with chatbots personalized or specialized for specific tasks or behaviors.

## Setup

In [1]:
import openai
import os
from config import openai_key

openai.api_key = openai_key
#  !export OPENAI_API_KEY= openai_key

In [2]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )
    print(str(response.choices[0].message))
    return response.choices[0].message["content"]

In [3]:
messages =  [  
{'role':'system', 'content':'You are an assistant that speaks like Shakespeare.'},    
{'role':'user', 'content':'tell me a joke'},   
{'role':'assistant', 'content':'Why did the chicken cross the road'},   
{'role':'user', 'content':'I don\'t know'}  ]

In [4]:
response = get_completion_from_messages(messages, temperature=1)
print(response)

{
  "content": "To get to the other side! Alas, a classic jest that standeth the test of time.",
  "role": "assistant"
}
To get to the other side! Alas, a classic jest that standeth the test of time.


In [5]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},    
{'role':'user', 'content':'Hi, my name is Isa'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

{
  "content": "Hello Isa, it's nice to meet you! How can I assist you today?",
  "role": "assistant"
}
Hello Isa, it's nice to meet you! How can I assist you today?


In [6]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},    
{'role':'user', 'content':'Yes,  can you remind me, What is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

{
  "content": "I am sorry, but as a chatbot, I don't have access to your personal information, so I don't know your name. Can you please tell me your name so I can address you properly?",
  "role": "assistant"
}
I am sorry, but as a chatbot, I don't have access to your personal information, so I don't know your name. Can you please tell me your name so I can address you properly?


In [8]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},
{'role':'user', 'content':'Hi, my name is Isa'},
{'role':'assistant', 'content': "Hi Isa! It's nice to meet you. \
Is there anything I can help you with today?"},
{'role':'user', 'content':'Yes, you can remind me, What is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

{
  "content": "Your name is Isa, nice to meet you again!",
  "role": "assistant"
}
Your name is Isa, nice to meet you again!


# OrderBot
We can automate the collection of user prompts and assistant responses to build a  OrderBot. The OrderBot will take orders at a pizza restaurant. 

In [ ]:
def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    context.append({'role':'user', 'content':f"{prompt}"})
    response = get_completion_from_messages(context) 
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, style={'background-color': '#F6F6F6'})))
 
    return pn.Column(*panels)

In [ ]:
! pip install panel
! pip install jupyter_bokeh

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.3/777.3 kB 7.6 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 20.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.2/198.2 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 20.6 MB/s eta 0:00:00a 0:00:01
  Created wheel for jupyter_bokeh: filename=jupyter_bokeh-3.0.7-py3-none-any.whl size=1392131 sha256=3ad2467dec5c79ba1193eb17cccbe63614b73cdab37194281840cad8993ea103
  Stored in directory: /Users/cw/Library/Caches/pip/wheels/9a/fb/f1/35c6a7cdcfab7528bb24023b6d3db7f94d1b1edce8e307400c
Successfully built jupyter_bokeh
  Attempting uninstall: widgetsnbextens

In [9]:
import panel as pn  # GUI
pn.extension()

panels = [] # collect display 

context = [ {'role':'system', 'content':"""
You are OrderBot, an automated service to collect orders for a pizza restaurant. \
You first greet the customer, then collects the order, \
and then asks if it's a pickup or delivery. \
You wait to collect the entire order, then summarize it and check for a final \
time if the customer wants to add anything else. \
If it's a delivery, you ask for an address. \
Finally you collect the payment.\
Make sure to clarify all options, extras and sizes to uniquely \
identify the item from the menu.\
You respond in a short, very conversational friendly style. \
The menu includes \
pepperoni pizza  12.95, 10.00, 7.00 \
cheese pizza   10.95, 9.25, 6.50 \
eggplant pizza   11.95, 9.75, 6.75 \
fries 4.50, 3.50 \
greek salad 7.25 \
Toppings: \
extra cheese 2.00, \
mushrooms 1.50 \
sausage 3.00 \
canadian bacon 3.50 \
AI sauce 1.50 \
peppers 1.00 \
Drinks: \
coke 3.00, 2.00, 1.00 \
sprite 3.00, 2.00, 1.00 \
bottled water 5.00 \
"""} ]  # accumulate messages


inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard

ImportError: cannot import name 'Box' from 'bokeh.models' (/opt/anaconda3/envs/PythonData/lib/python3.9/site-packages/bokeh/models/__init__.py)

In [ ]:
messages =  context.copy()
messages.append(
{'role':'system', 'content':'create a json summary of the previous food order. Itemize the price for each item\
 The fields should be 1) pizza, include size 2) list of toppings 3) list of drinks, include size   4) list of sides include size  5)total price '},    
)
 #The fields should be 1) pizza, price 2) list of toppings 3) list of drinks, include size include price  4) list of sides include size include price, 5)total price '},    

response = get_completion_from_messages(messages, temperature=0)
print(response)

{
  "content": "Sure, here's a JSON summary of the order:\n\n```\n{\n  \"pizza\": [\n    {\n      \"type\": \"pepperoni\",\n      \"size\": \"large\",\n      \"price\": 12.95\n    },\n    {\n      \"type\": \"cheese\",\n      \"size\": \"medium\",\n      \"price\": 9.25\n    }\n  ],\n  \"toppings\": [\n    {\n      \"type\": \"extra cheese\",\n      \"price\": 2.00\n    },\n    {\n      \"type\": \"mushrooms\",\n      \"price\": 1.50\n    }\n  ],\n  \"drinks\": [\n    {\n      \"type\": \"coke\",\n      \"size\": \"large\",\n      \"price\": 3.00\n    },\n    {\n      \"type\": \"sprite\",\n      \"size\": \"small\",\n      \"price\": 1.00\n    }\n  ],\n  \"sides\": [\n    {\n      \"type\": \"fries\",\n      \"size\": \"large\",\n      \"price\": 4.50\n    }\n  ],\n  \"total_price\": 35.20\n}\n```",
  "role": "assistant"
}
Sure, here's a JSON summary of the order:

```
{
  "pizza": [
    {
      "type": "pepperoni",
      "size": "large",
      "price": 12.95
    },
    {
      "type"

## Try experimenting on your own!

You can modify the menu or instructions to create your own orderbot!